Just for developing the CryoEnv2

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import cait as ai
import cait.versatile as vai

# Get multiprocessing
n_processes = ai._available_workers
# Standard values for plotting
backend = "plotly" # Choose "mpl" for keeping plots when closing, "plotly" for interactive plotting

# Go up one level because notebook is in folder
import sys
import os
sys.path.append(os.path.abspath("."))  # Go to project root folder

# New cryoEnv implementation
from src.CryoEnv2 import CryoEnv as cryo

# Time and cryo environment definition
First of all, it is important to define the time vector for one testpulse event.
This should be kept similar to the events in cait, so we use the same record_length.

In [2]:
record_length = 2**14       # Standard value in cait
sample_frequency = 50e3     # Standard value in vdaq
dt = 1 / sample_frequency   # Time step

time = np.linspace(0, record_length/sample_frequency, record_length)  # Time vector
print("Maximum time is: ", np.max(time), "s")

Maximum time is:  0.32768 s


In [3]:
# Initialize cryo
test_env = cryo(record_length=record_length, sample_frequency=sample_frequency)

Cryogenic environment initialized.


# Filling the ETM with life
First of all, the cryo environment has to be created.

In [4]:
# Create dict for all relevant electrothermal model parameters (2-comp model)
etm_params_2C = {
    "C_e": 10e-3,     # Heat capacity electrical system
    "C_a": 10e-3,     # Heat capacity absorber
    "G_ea": 1e-3,     # Thermal conductance between electrical system and absorber
    "G_ab": 1e-3,     # Thermal conductance between absorber and bath
    "G_eb": 1e-3,     # Thermal conductance between electrical system and bath
    "T_b": 0.1,      # Bath temperature
    "R_S": 1e-3,      # Shunt resistance
    "L": 400e-9      # Inductance from SQUID input coil
}

# Initialize electrothermal model
test_env.init_etm(n_components=2, params=etm_params_2C) # give number of components, otherwise parameters will not fit

2-component Electrothermal model initialized.


# Define Heater
The DAC is the voltage value set to the Heater.
This is usually a constant DC, which is superimposed with Testpulses.
The constant part sets the OP of the detector.

Before using ´apply_testpulse´, the DC heater voltage (which defines the operating point) needs to be set.

In [6]:
# like for the etm parameter, also here can be a very short dictionary be defined
heater_params = {
    "R_H": 10e-3  # Typical resistance value: 10 mOh
}

# Before we can use the heater, we have to initialize it
test_env.init_heater(params=heater_params)

# Additionally, we have to set an OP by choosing a constant DC value for it
test_env.set_heater_voltage(5.0)  # Set heater voltage to 5 V

Heater initialized.
Heater voltage set to 5.0 V.
